# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [59]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import collections
import pandas as pd

import cv2

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


## Env setup

In [60]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("/Users/vipul/virtualenv/tensorflow/lib/python2.7/site-packages/tensorflow/models/research/object_detection/")

## Object detection imports
Here are the imports from the object detection module.

In [61]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [62]:
# the video feed
cap = cv2.VideoCapture("cola_1x_b2.avi")


# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = 'frozen_inference_graph_v009.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'label_map_v009.pbtxt')

NUM_CLASSES = 2

## Download Model

In [63]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph_v009.pb' in file_name:
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [64]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [65]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print category_index

{1: {'id': 1, 'name': u'cola'}, 2: {'id': 2, 'name': u'koko'}}


## Helper code

In [66]:
def shout(label, count):
    if count == 1:
        os.system("say %s"%label)
        return
    if count > 1:
        os.system("say %d, %s"%(count,label))
    

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def get_detection_values(image,
                        boxes,
                        classes,
                        scores,
                        category_index,
                        instance_masks=None,
                        keypoints=None,
                        use_normalized_coordinates=False,
                        max_boxes_to_draw=20,
                        min_score_thresh=.5,
                        agnostic_mode=False,
                        line_thickness=4):

  threshold_value = 0.9
  box_to_display_str_map = collections.defaultdict(list)
  box_to_color_map = collections.defaultdict(str)
  box_to_instance_masks_map = {}
  box_to_keypoints_map = collections.defaultdict(list)
    
  results = []
  if not max_boxes_to_draw:
    max_boxes_to_draw = boxes.shape[0]
  
#   print(boxes.shape)
    
  bound = min(max_boxes_to_draw, boxes.shape[0])
#   print("bound - %d"%bound)
#   print "Scores"
#   print(scores)
  # why range is min of boxes? What about rest of the indexes? => Scores are sorted in descending order.
  for i in range(bound):
    if scores is None or scores[i] > threshold_value:
      box = tuple(boxes[i].tolist())
      if instance_masks is not None:
        box_to_instance_masks_map[box] = instance_masks[i]
      if keypoints is not None:
        box_to_keypoints_map[box].extend(keypoints[i])
      if scores is None:
        box_to_color_map[box] = 'black'
      else:
        if not agnostic_mode:
          if classes[i] in category_index.keys():
            class_name = category_index[classes[i]]['name']
#             print("Box")
#             print(box)
          else:
            class_name = 'N/A'
        else:
          pass
    else:
        class_name = None
#         print "Skipped"
        
#     print class_name
#     print scores[i]
#     print box
    if class_name is not None:
        results.append([class_name, scores[i], box])
  
  if not results:
        return None
  df = pd.DataFrame(results)
  df.columns = ["Class", "Score", "Box"]

# # Filter by threshold value
#   df = df.loc[df['Score'] > threshold_value]
  return df
        


# Detection

In [67]:

# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'

TEST_IMAGE_PATHS = []

# creating a list of all image paths
for f in os.listdir(PATH_TO_TEST_IMAGES_DIR):
    if f.endswith(".jpg"):
        TEST_IMAGE_PATHS.append(os.path.join(PATH_TO_TEST_IMAGES_DIR, f))

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [68]:
# All the tagged images(after detection) will be written to this directory
OUTPUT_IMG_DIR = "test_ouput_images"
if not os.path.exists(OUTPUT_IMG_DIR):
    os.makedirs(OUTPUT_IMG_DIR)
    
RAW_IMG_DIR = "raw_images"
if not os.path.exists(RAW_IMG_DIR):
    os.makedirs(RAW_IMG_DIR)

    
# Write box coordinate to csv file
f = open('boxes.csv','w+')
f.write("Format => imagename.jpg, ymin, xmin, ymax, xmax")
f.write('\n')
    
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    
    count = 0
    make_sound = True
    while True:
      ret, image_np = cap.read()
      
      if image_np is None or image_np.size ==0:
        break
        
      filename = "image%d.jpg"%count
      count = count + 1
      cv2.imwrite(RAW_IMG_DIR + "/" + filename, image_np)
    
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection.
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
        
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
    
      
      cv2.imwrite(OUTPUT_IMG_DIR + "/" + filename, image_np)
#       plt.imsave(OUTPUT_IMG_DIR + "/" + filename, image_np)
#       print(image_np.shape)
    
      df = get_detection_values(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
        
      height = image_np.shape[0]
      width = image_np.shape[1]
        
      if df is not None:
          print filename
          print("box")
          box = df.iloc[0]['Box']
          print(box[0] * height)
          print(box[1] * width)
          print(box[2] * height) 
          print(box[3] * width)
          line = "%s, %d, %d, %d, %d"%(filename, box[0] * height, box[1] * width, box[2] * height, box[3] * width)
          print line
          f.write(line)
          f.write('\n')
        
#       print(df)
        # Using text to speech to voice out the label
      if (make_sound or count %5 ==0) and df is not None:
        for label in df.Class.unique():
            print label
            shout(label, df[df.Class == label].shape[0])
        make_sound = False
        
      print("+1")
      cv2.imshow("video-feed", cv2.resize(image_np, (1920, 1080)))
      
      if(cv2.waitKey(25) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break
f.close()            

image0.jpg
box
495.87364912
1526.98677063
870.05764246
1746.84516907
image0.jpg, 495, 1526, 870, 1746
cola
+1
image1.jpg
box
479.956068993
1237.87055969
867.972664833
1469.96520996
image1.jpg, 479, 1237, 867, 1469
+1
image2.jpg
box
467.409768105
1182.81932831
856.170258522
1432.26436615
image2.jpg, 467, 1182, 856, 1432
+1
image3.jpg
box
476.353497505
1159.80102539
850.665206909
1381.81274414
image3.jpg, 476, 1159, 850, 1381
+1
image4.jpg
box
462.435278893
1121.36959076
864.097795486
1353.35140228
image4.jpg, 462, 1121, 864, 1353
cola
+1
image5.jpg
box
464.363315105
1084.61196899
846.688306332
1298.15116882
image5.jpg, 464, 1084, 846, 1298
+1
image6.jpg
box
478.528854847
1053.53588104
856.781995296
1267.01900482
image6.jpg, 478, 1053, 856, 1267
+1
image7.jpg
box
459.760901928
992.920875549
858.225367069
1234.82906342
image7.jpg, 459, 992, 858, 1234
+1
image8.jpg
box
474.848520756
960.761947632
849.320132732
1180.02754211
image8.jpg, 474, 960, 849, 1180
+1
image9.jpg
box
474.698338509
90